In [1]:
import nltk
nltk.download('punkt')       
nltk.download('stopwords') 
nltk.download('punkt_tab')  
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import numpy as np
import pandas as pd
from helper_functions import *
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/rebekaheichberg/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data

In [2]:
# Load in Recall data

# Paths to all three files
recall_files = [
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_ArtsandCrafts.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Riding_Toys.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Toys.csv"
]

recall_dfs = [load_clean_csv(path) for path in recall_files]
recalls_df = pd.concat(recall_dfs, ignore_index=True)



In [3]:
# pre process the text
recalls_df['lemmas'] = recalls_df['Incident Description'].apply(preprocess)

In [4]:
!pip install yake

In [5]:
import yake

In [8]:
kw_extractor = yake.KeywordExtractor(lan="en", n=1, top=5)

In [9]:
def extract_keywords_from_lemmas(lemmas_list, top_n=5):
    text = " ".join(lemmas_list)
    kw_tuples = kw_extractor.extract_keywords(text)
    # If you only want the keywords themselves:
    keywords = [kw for kw, score in kw_tuples][:top_n]
    return keywords

# Apply to each row, storing the resulting keyword list in a new column:
recalls_df["keywords"] = recalls_df["lemmas"].apply(lambda lem: extract_keywords_from_lemmas(lem))

recalls_df

,Report No.,Report Date,Sent to Manufacturer / Importer / Private Labeler,Publication Date,Category of Submitter,Product Description,Product Category,Product Sub Category,Product Type,Product Code,...,Damage Description,Damage Repaired,Product Was Modified Before Incident,Have You Contacted The Manufacturer,If Not Do You Plan To,Answer Explanation,Company Comments,Associated Report Numbers,lemmas,keywords
0,20231002-D13C7-2147344911,10/2/2023,10/27/2023,11/20/2023,Consumer,Slime globe with colored spheres which resembl...,Toys & Children,Arts & Crafts,Molding Compounds (1376),1376,...,NaN,NaN,NaN,NaN,NaN,NaN,TOYSMITH: Thank you for the opportunity to rev...,NaN,"[receive, toy, retail, store, polka, dot, slim...","[receive, redact, child, toy, retail]"
1,20180126-BBF18-2147393362,1/26/2018,4/13/2018,4/27/2018,Consumer,Slime kit from Nickelodeon by Cra-Z-Art,Toys & Children,Arts & Crafts,Molding Compounds (1376),1376,...,NaN,NaN,No,Yes,NaN,I already have explained.,We at Cra-Z-Art® are very sorry that the cons...,NaN,"[year, old, nephew, custody, receive, gift, ch...","[toddler, call, product, company, ingredient]"
2,20141231-7C153-2147437145,12/31/2014,1/9/2015,1/26/2015,Consumer,Lalaloopsy Color Me Doll ( Squiggles N. Shapes...,Toys & Children,Arts & Crafts,Crayons or Chalk (5010),5010,...,NaN,NaN,NaN,No,No,NaN,The chalk markers included with this item are ...,NaN,"[buy, toy, daughter, toy, r, u, lalaloopsy, co...","[allergic, reaction, buy, immediately, toy]"
3,20150204-EC795-2147435743,2/4/2015,3/11/2015,4/1/2015,Health Care Professional,Play Doh by Hasbro,Toys & Children,Arts & Crafts,Molding Compounds (1376),1376,...,NaN,NaN,NaN,Yes,NaN,"[REDACTED] , [REDACTED] at hasbro, was contacted","At Hasbro, the safety of our products has alw...",20150204-60300-2147435758,"[six, year, old, girl, ingest, approximately, ...","[level, sodium, calcium, year, girl]"
4,20210313-4AFB1-2147365371,3/13/2021,3/24/2021,4/7/2021,Consumer,Cra-z-art Nickelodeon slime party pack,Toys & Children,Arts & Crafts,Molding Compounds (1376),1376,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,"[buy, nickelodeon, slime, party, pack, five, w...","[buy, treatment, slime, skin, nickelodeon]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2509,20250330-F4BB1-2147332207,3/30/2025,4/7/2025,4/21/2025,Consumer,It is a reversible stuffed animal called a fli...,Toys & Children,Toys,"Dolls, Plush Toys, and Action Figures (1394)",1394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[year, old, year, old, daughter, put, stuffed,...","[year, head, face, product, suffocation]"
2510,20250414-9F970-2147331938,4/14/2025,4/29/2025,5/13/2025,Consumer,Bubble blaster,Toys & Children,Toys,"Toys, Not Elsewhere Classified (1381)",1381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[place, battery, product, get, two, aa, batter...","[battery, place, product, require, start]"
2511,20250112-CF901-2147333938,1/12/2025,1/15/2025,2/6/2025,Consumer,Magnetic ferrite putty stones 500+ in a tin so...,Toys & Children,Toys,Building Sets (1345),1345,...,NaN,NaN,NaN,NaN,NaN,NaN,Retrospective Goods LLC: Crags are adult produ...,NaN,"[year, old, daughter, swallow, ferrite, putty,...","[year, warning, daughter, swallow, ferrite]"
2512,20250426-11C99-2147331693,4/26/2025,5/1/2025,5/15/2025,Consumer,Magnetic building tiles by AnbaluluMagnetic Ti...,Toys & Children,Toys,Building Sets (1345),1345,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,"[year, old, magnetic, building, tile, notice, ...","[magnet, tile, notice, young, child]"


In [ ]:
recalls_df['keywords'].to_clipboard()